In [1]:
from __future__ import print_function

import pygatt
import numpy as np
import struct
import time as t

import matplotlib.pyplot as plt
%matplotlib notebook

from ipywidgets import widgets, interact, IntSlider
from IPython.display import display
from IPython.display import clear_output

In [2]:
#configuration

HANDLE = {
        'NAME':          0x0003,
        'DATA':          0x003C,
        'CONFIG':        0x003F,
        'PERIOD':        0x0041,
        'NOTIFICATION':  0x003D}

#Gyroscope z axis enable
zGyroEnable = 2**0
#Gyroscope y axis enable
yGyroEnable = 2**1
#Gyroscope x axis enable
xGyroEnable = 2**2

#Accelerometer z axis enable
zAccEnable = 2**3
#Accelerometer y axis enable
yAccEnable = 2**4
#Accelerometer x axis enable
xAccEnable = 2**5
#Magnetometer enable (all axes)
MagnEnable = 2**6
#Wake-On-Motion Enable
OnMotrionEnable = 2**7
#Accelerometer range (0=2G, 1=4G, 2=8G, 3=16G) bits 8:9
AccRange2G = 0
AccRange4G = 2**1
AccRange8G = 2**2
AccRange16G = 2**1 | 2**2

In [3]:
# adapter = pygatt.GATTToolBackend()

In [4]:
# device = adapter.connect('54:6c:0e:79:5b:86', timeout=5)

In [5]:
class myBLE:
    def __init__(self, MAC):
        self.MAC = MAC
        # define PC bluetooth adapter
        self.adapter = pygatt.GATTToolBackend()
    
    def connect(self, timeout = 5):
        self.adapter.start()
        self.myDevice = self.adapter.connect(self.MAC, timeout=timeout)
        
    def readHandle(self, handle, HEXA = False):
        # return data array
        data = self.myDevice.char_read_handle(hex(handle))
        size = len(data)
        if not HEXA:
            return struct.unpack(str(size)+'B', data)
        else:
            return data
        
    def writeHandle(self, handle, value):
        self.myDevice.char_write_handle(handle, bytearray(value))
        

In [6]:
#connect BLE
texasBLE = myBLE('54:6c:0e:79:5b:86')

In [7]:
texasBLE.connect()

In [8]:
#Notification enable
texasBLE.writeHandle(HANDLE['NOTIFICATION'],[0x01,0x00])

In [9]:
texasBLE.writeHandle(HANDLE['PERIOD'],[0xFF])

In [10]:
#Config
texasBLE.writeHandle(HANDLE['CONFIG'],
    [( OnMotrionEnable | xAccEnable),
     0x00 ])

In [11]:
def getHandle(hnd):
    data = texasBLE.readHandle(hnd)
    print("Data {}".format(data))
    print(bytearray(data))

interact(getHandle, hnd=HANDLE);

interactive(children=(Dropdown(description='hnd', options={'NAME': 3, 'DATA': 60, 'CONFIG': 63, 'PERIOD': 65, …

In [38]:
#print data
t0 = t.time()
for i in range(0,100):
    clear_output(wait=True)
    print("Count {}, \nvalue {}".format(i,texasBLE.readHandle(HANDLE['DATA'], HEXA = True)))
print("Final time %0.2f s"% (t.time()-t0))


Count 99, 
value bytearray(b'\x00\x00\x00\x00\x00\x00A\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
Final time 14.13 s


In [49]:
texasBLE.adapter.stop()

In [ ]:
"Dynamic plotting in matplotlib. Copy and paste into a Jupyter notebook."
# written October 2016 by Sam Greydanus

def plt_dynamic(x, y, ax, colors=['b']):
    for color in colors:
        ax.plot(x, y, color)
    fig.canvas.draw()
    
fig,ax = plt.subplots(1,1)
ax.set_xlabel('X') ; ax.set_ylabel('Y')
ax.set_xlim(0,100) ; ax.set_ylim(-500,500)
xs, ys = [], []

# this is any loop for which you want to plot dynamic updates.
# in my case, I'm plotting loss functions for neural nets
for x in range(100):
    value = texasBLE.readHandle(HANDLE['DATA'])
    y = value[4]
    xs.append(x)
    ys.append(y)
    if x % 5 == 0:
        plt_dynamic(xs, ys, ax)
plt_dynamic(xs, ys, ax)